In [10]:
import random
import copy
import math
import numpy as np
import pyspark
import csv

inn_numbers = 0
inovatii_generatii = {}

def sigmoid(x):
    x = np.array([x])
    return np.mean(1/(1+ np.exp(-4.9 * x)))

class Gena:
    def __init__(self, nume, tip):
        self.nume = nume
        self.tip = tip
    def __eq__(self, gena1):
        if self.nume == gena1.nume:
            return True
        return False
    def __hash__(self):
        return self.nume
    def __str__(self):
        return str(self.nume)
    def __lt__(self, gena_aux):
        if self.nume < gena_aux.nume:
            return True
        else:
            return False

class Conexiune:
    def __init__(self, inp, out, wgh):
        self.input = inp
        self.output = out
        self.weight = wgh
        self.status = 1
        self.innovation_number = self.determinare_inovatie(inp, out)
    def __str__(self):
        return str(self.input) + ", " + str(self.output) + ", " + str(self.weight) + ", " + str(self.innovation_number) + ", " + str(self.status) + " - "
    def __eq__(self, conn):
        if self.innovation_number == conn.innovation_number:
            return True
        else:
            return False
    def __hash__(self):
        return self.innovation_number
    def determinare_inovatie(self, inp, out):
        global inovatii_generatii
        global inn_numbers
        if (inp, out) in inovatii_generatii.values():
            for i in inovatii_generatii:
                if inovatii_generatii[i] == (inp, out):
                    return i
        else:
            inn_numbers += 1
            inovatii_generatii[inn_numbers] = (inp, out)
            return inn_numbers

class Genom:
    def __init__(self, gene, conexiuni):
        self.gene = copy.deepcopy(gene)
        self.conexiuni = copy.deepcopy(conexiuni)
        self.fitness = self.feed_forwoard()
        
    def __str__(self):
        text = "Gene: \n"
        for i in self.gene:
            text = text + str(i) + "; "
        text += "\nConexiuni: \n"
        for i in self.conexiuni:
            text = text + str(i) + "; "
        return text
    
    
    def eliminare_disabled(self):
        de_sters = [n for n in self.conexiuni if n.status == 0]
        for conn in de_sters:
            self.conexiuni.remove(conn)

    
    def adaugare_conexiune(self, i, j):
        verificare_con = 0
        maxim = 0
        for k in range(0, len(self.conexiuni)):
            if self.conexiuni[k].input == i and self.conexiuni[k].output == j:
                if self.conexiuni[k].status == 0:
                    verificare_con = 1
                    maxim = k
                else:
                    return
        if verificare_con == 1:
            self.conexiuni[maxim].status = 1
        else:
            con = Conexiune(i, j, random.uniform(0, 1)*random.choice([-1, 1]))
            self.conexiuni += [con]
        
    def adaugare_gena(self, gena):
        self.gene += [gena]
        
    def modificare_weighturi(self):
        conexiune = random.choice(self.conexiuni)
        verificare = random.uniform(0, 1)
        if verificare <= 0.8:
            conexiune.weight += random.uniform(0, 1) * random.choice([-1, 1])
        else:
            conexiune.weight = random.uniform(0, 1)*random.choice([-1, 1])

            
    def adaugare_nod(self):
        nume_genomi = [i.nume for i in self.gene]
        conexiuni_valide = [i for i in self.conexiuni if i.status]
        if len(conexiuni_valide) == 0:
            return
        conexiune = random.choice(conexiuni_valide)
        conexiune.status = 0
        gena_noua = Gena(max(nume_genomi) + 1, 2)
        conexiune_noua1 = Conexiune(conexiune.input, gena_noua, 1)
        conexiune_noua2 = Conexiune(gena_noua ,conexiune.output, conexiune.weight)
        self.gene += [gena_noua]
        self.conexiuni += [conexiune_noua1, conexiune_noua2]
        
    def mutare(self):
        verificare = random.randint(0, 100)
        if verificare < 90:
            self.modificare_weighturi()
        verificare = random.randint(0, 100)
        if verificare < 5:
            conex = [(i.input, i.output) for i in self.conexiuni]
            pereche = self.random_nodes()
            if pereche not in conex and (pereche[1], pereche[0]) not in conex:
                self.adaugare_conexiune(*pereche)
                
        verificare = random.randint(0, 100)
        if verificare < 3:
            self.adaugare_nod()

            
    def random_nodes(self):
        i = random.choice([n for n in self.gene if n.tip != 1])
        j_list = [n for n in self.gene if n.tip != 0]
        if len(j_list) == 0:
            nume = [n.nume for n in self.gene]
            j = Gena(max(nume) + 1, 2)
            self.adaugare_nod()
        else:
            j = random.choice(j_list)
        return (i, j)
    
    
    def feed_forwoard(self):
        suma = 0
        teste = []
        """
        with open('C:\\Users\\MrSpV\\Desktop\\data.csv') as csvfile:
            readCSV = csv.reader(csvfile, delimiter=',')
            for row in readCSV:
                if row[0] == 'id':
                    continue
                if row[1] == 'M':
                    teste = teste + [([1] + list(map(lambda x: float(x), row[2:])),[1])]
                else:
                    teste = teste + [([1] + list(map(lambda x: float(x), row[2:])),[0])]
        teste = teste[1:]
        teste = teste[35:40]
        """
        teste = [([1, 14.25, 21.72, 93.63, 633.0, 0.09823, 0.1098, 0.1319, 0.05598, 0.1885, 0.06125, 0.286, 1.019, 2.657, 24.91, 0.005878, 0.02995, 0.04815, 0.01161, 0.02028, 0.004022, 15.89, 30.36, 116.2, 799.6, 0.1446, 0.4238, 0.5186, 0.1447, 0.3591, 0.1014], [1]), ([1, 13.03, 18.42, 82.61, 523.8, 0.08983, 0.03766, 0.02562, 0.02923, 0.1467, 0.05863, 0.1839, 2.342, 1.17, 14.16, 0.004352, 0.004899, 0.01343, 0.01164, 0.02671, 0.001777, 13.3, 22.81, 84.46, 545.9, 0.09701, 0.04619, 0.04833, 0.05013, 0.1987, 0.06169], [0]), ([1, 14.99, 25.2, 95.54, 698.8, 0.09387, 0.05131, 0.02398, 0.02899, 0.1565, 0.05504, 1.214, 2.188, 8.077, 106.0, 0.006883, 0.01094, 0.01818, 0.01917, 0.007882, 0.001754, 14.99, 25.2, 95.54, 698.8, 0.09387, 0.05131, 0.02398, 0.02899, 0.1565, 0.05504], [1]), ([1, 13.48, 20.82, 88.4, 559.2, 0.1016, 0.1255, 0.1063, 0.05439, 0.172, 0.06419, 0.213, 0.5914, 1.545, 18.52, 0.005367, 0.02239, 0.03049, 0.01262, 0.01377, 0.003187, 15.53, 26.02, 107.3, 740.4, 0.161, 0.4225, 0.503, 0.2258, 0.2807, 0.1071], [1]), ([1, 13.44, 21.58, 86.18, 563.0, 0.08162, 0.06031, 0.0311, 0.02031, 0.1784, 0.05587, 0.2385, 0.8265, 1.572, 20.53, 0.00328, 0.01102, 0.0139, 0.006881, 0.0138, 0.001286, 15.93, 30.25, 102.5, 787.9, 0.1094, 0.2043, 0.2085, 0.1112, 0.2994, 0.07146], [1])]
        #teste = [([1,1,1], [0]), ([1,1,0], [1]), ([1,0,1], [1]), ([1,0,0], [0])]
        for test in teste:
            rezultat = self.ff_util(test[0])
            rezultat = np.mean(np.array(test[1])) - rezultat
            rezultat = rezultat * rezultat
            suma += rezultat
        return suma / 4
    
    def ff(self, inputs):
        valori = {}
        valori1 = {}
        for i in self.gene:
            valori[i.nume] = 0
            valori1[i.nume] = 0
        inp = [n for n in self.gene if n.tip == 0]
        inp.sort()
        for i in range(0, len(inp)):
            valori[inp[i].nume] += inputs[i]
        for conex in self.conexiuni:
            valori[conex.output.nume] += sigmoid(valori[conex.input.nume] * conex.weight)

        for conex in self.conexiuni:
            valori1[conex.output.nume] += sigmoid(valori[conex.input.nume] * conex.weight)
            
        #print(valori)
        out = [n for n in self.gene if n.tip == 1]
        return valori1[out[0].nume]
        
    
    def ff_util(self, inputs):
        valori = []
        inp = [n for n in self.gene if n.tip == 0]
        inp.sort()
        coada = [n for n in self.gene if n.tip == 1]
        for i in range(0, len(inp)):
            valori += [(inp[i], inputs[i] * 0.001)]
        ttl = 2000
        while coada != []:
            if ttl == 0:
                return 1000
            ttl -= 1
            nod = coada.pop()
            if nod.tip == 0:
                continue
            adiacente = []
            conexiuni_adiacente = []
            for i in self.conexiuni:
                if i.output.nume == nod.nume and i.status == 1:
                    adiacente += [i.input]
                    conexiuni_adiacente += [i]
            posibile = [n.nume for n in coada]
            for i in adiacente:
                if i.nume not in posibile:
                    coada += adiacente
            ok = 0
            suma = 0
            for i in range(0, len(adiacente)):
                ok1 = 0
                for j in valori:
                    if j[0].nume == adiacente[i].nume:
                        suma =  suma + j[1] * conexiuni_adiacente[i].weight
                        ok1 = 1
                        break
                if ok1 == 0:
                    ok = 1
                    break
            if ok == 1:
                coada = [nod] + coada
            else:
                ok2 = 0
                for i in range(0, len(valori)):
                    if valori[i][0].nume == nod.nume:
                        ok2 = i
                        break
                if ok2 == 0:
                    valori += [(nod, sigmoid(suma))]
                else:
                    valori[i] = (nod, sigmoid(suma))
        return np.mean(np.array([n[1] for n in valori if n[0].tip == 1]))
    
    def feed_forwoard_util(self, inputs):
        valoare_noduri = []
        coada = []
        inputuri = [n for n in self.gene if n.tip == 0]
        coada = [n for n in self.gene if n.tip == 1]
        for i in range(0, len(inputuri)):
            valoare_noduri += [(inputuri[i], inputs[i])]
        while coada != []:
            nod = coada.pop()
            if nod.tip == 0:
                continue
            adiacente = []
            conexiuni_adiacente = []
            for i in self.conexiuni:
                if i.output.nume == nod.nume:
                    adiacente += [i.input]
                    conexiuni_adiacente += [i]
            coada += adiacente
            coada = list(set(coada))
            ok = 0
            suma = 0
            for i in range(0, len(adiacente)):
                ok1 = 0
                for j in valoare_noduri:
                    if j[0].nume == adiacente[i].nume:
                        suma += j[1] * conexiuni_adiacente[i].weight
                        ok1 = 1
                        break
                if ok1 == 0:
                    ok = 1
                    break
            if ok == 1:
                coada = [nod] + coada
            else:
                ok2 = 0
                for i in range(0, len(valoare_noduri)):
                    if valoare_noduri[i][0].nume == nod.nume:
                        ok2 = i
                        break
                if ok2 == 0:
                    valoare_noduri += [(nod, suma)]
                else:
                    valoare_noduri[i] = (nod, suma)
        return np.mean(np.array([n[1] for n in valoare_noduri if n[0].tip == 1]))
    
    def clonare(self):
        clona = Genom(self.gene, self.conexiuni)
        return clona

 
class Specie:
    def __init__(self, genomi):
        self.genomi = copy.deepcopy(genomi)
        self.max_fitness = self.determinare_campion()
        self.max_generatie = self.determinare_campion()
        self.generatie_curenta = 1
        self.moarta = 0
        
    def determinare_superior(self):
        maxim = -100
        for genom in self.genomi:
            fitness = genom.fitness
            if fitness > maxim:
                maxim = fitness
        if maxim > self.max_fitness:
            self.max_fitness = maxim
            self.max_generatie = self.generatie_curenta
            
            
    def determinare_campion(self):
        maxim = -100
        for genom in self.genomi:
            fitness = genom.fitness
            if fitness > maxim:
                maxim = fitness
        return maxim
    
    def determinare_existenta(self):
        if self.generatie_curenta == 1:
            return True
        fitness = self.determinare_campion()
        self.determinare_superior()
        fit = fitness  != self.max_fitness
        end = self.generatie_curenta != self.max_generatie
        return fit and end
    
    def breed(self):
        if random.uniform(0, 1) < 0.4 or len(self.genomi) == 1:
            child = random.choice(self.genomi).clonare()
            child.mutare()
        else:
            mom = random.choice(self.genomi)
            dad = random.choice([i for i in self.genomi if i != mom])
            if mom.fitness < dad.fitness:
                child = self.crossover(mom, dad)
            else:
                child = self.crossover(dad, mom)
        return child
    
            
    def eliminare_genomi(self, tip):
        ok = 1
        ttl = 300
        while ok == 1 and ttl > 0:
            ttl -= 1
            ok = 0
            for i in range(0, len(self.genomi) - 1):
                if self.genomi[i].fitness > self.genomi[i + 1].fitness:
                    self.genomi[i], self.genomi[i + 1] = self.genomi[i + 1], self.genomi[i]
                    ok = 1
        if tip == 1:
            self.genomi = self.genomi[0]
        else:
            self.genomi = self.genomi[:math.ceil(len(self.genomi) / 4)]
        
    def fitness_specie(self):
        suma = 0
        for i in self.genomi:
            suma += i.fitness
        return suma
    
    def genom_maxim(self):
        maxim = 1000
        genom_maxim = 0
        for genom in self.genomi:
            if genom.fitness < maxim:
                maxim = genom.fitness
                genom_maxim = genom
        return (maxim, genom_maxim)
    
    def crossover(self, genom1, genom2):
        lista_gene = []
        for i in genom1.gene:
            if i not in genom2.gene:
                lista_gene += [i]
        lista_gene += genom2.gene
        conexiuni = []
        for i in genom1.conexiuni:
            ok = 0
            for j in genom2.conexiuni:
                if i.input == j.input and i.output == j.output:
                    if random.choice([0, 1]) == 0:
                        conexiuni.append(copy.deepcopy(i))
                    else:
                        conexiuni.append(copy.deepcopy(j))
                    ok = 1
                    break
            if ok == 0:
                conexiuni.append(i)
        de_sters = []
        for i in lista_gene:
            if i.nume not in [n.input.nume for n in conexiuni] + [n.output.nume for n in conexiuni]:
                de_sters += [i]
        for i in de_sters:
            lista_gene.remove(i)
        copil = Genom(lista_gene, conexiuni)
        return copil
    
    def distanta_genomi(self, genom1, genom2):
        numere1 = [n.innovation_number for n in genom1.conexiuni]
        numere2 = [n.innovation_number for n in genom2.conexiuni]
        maxn1 = max(numere1)
        maxn2 = max(numere2)
        prag = 3
        d = len([n for n in numere1 if n not in numere2 and n < maxn2] + [n for n in numere2 if n not in numere1 and n < maxn1])
        e = len([n for n in numere1 if n not in numere2 and n > maxn2] + [n for n in numere2 if n not in numere1 and n > maxn1])
        c = 0.4
        n = max(len(numere1), len(numere2))
        w = abs(genom1.fitness - genom2.fitness)
        if (d + e) / n + c * w < prag:
            return 1
        else:
            return 0
        

        
class Specii:
    def __init__(self, spe):
        self.spec = spe
        
    def update_fitness(self):
        for specie in self.spec:
            if specie.genom_maxim()[0] < specie.max_fitness:
                specie.max_fitness = specie.genom_maxim()[0]
                specie.max_generatie = specie.generatie_curenta
                
    def eliminare_specii(self):
        genab = Gena(0, 0)
        gena1 = Gena(1, 0)
        gena2 = Gena(2, 0)
        gena3 = Gena(3, 2)
        gena0 = Gena(5, 2)
        gena4 = Gena(4, 1)
        gena5 = Gena(4, 1)
        obj =  Conexiune(gena1, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
        ob2 =  Conexiune(gena2, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
        ob3 =  Conexiune(genab, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
        gen1 = Genom([genab,gena1, gena2, gena4], [obj, ob2, ob3])

        lista1 = []
        for _ in range(0, 15):
            lista1.append(copy.deepcopy(gen1))
        self.update_fitness()
                
        de_sters = []
        for i in range(0, len(self.spec)):
            if self.spec[i].generatie_curenta - self.spec[i].max_generatie >= 15 or self.spec[i].genom_maxim() == 1000:
                de_sters += [i]
        for i in de_sters:
            self.spec[i].genomi = lista1
                
        
    def clasificare_genom(self, genom):
        start_time = time.time()
        if len(self.spec) == 0:
            self.spec += [Specie([genom])]
        else:
            for specie in self.spec:
                if len(specie.genomi) == 0:
                    continue
                if len(specie.genomi) < 15:
                    specie.genomi += [genom]
                    #print("--- %s por RIO ---" % (time.time() - start_time))
                    return
            self.spec += [Specie([genom])]
            
    def distanta_genomi(self, genom1, genom2):
        numere1 = [n.innovation_number for n in genom1.conexiuni]
        numere2 = [n.innovation_number for n in genom2.conexiuni]
        maxn1 = max(numere1)
        maxn2 = max(numere2)
        prag = 3
        d = len([n for n in numere1 if n not in numere2 and n < maxn2] + [n for n in numere2 if n not in numere1 and n < maxn1])
        e = len([n for n in numere1 if n not in numere2 and n > maxn2] + [n for n in numere2 if n not in numere1 and n > maxn1])
        c = 0.4
        n = max(len(numere1), len(numere2))
        w = abs(genom1.fitness - genom2.fitness)
        if (d + e) / n + c * w < prag:
            return 1
        else:
            return 0
        
    
    def eliminare_underfit_specii(self, tip):
        for i in self.spec:
            i.eliminare_genomi(tip)
    
    def fitness_specii(self):
        suma = 0
        for i in self.spec:
            suma += i.fitness_specie()
        return suma
    
    def determinare_populatie(self):
        return sum([len(n.genomi) for n in self.spec])
    
    def specie_maximala(self):
        minim = 1000
        specie_minima = 0
        genom_minim = 0
        for specie in self.spec:
            if len(specie.genomi) == 0:
                continue
            if specie.genom_maxim()[0] < minim:
                minim = specie.genom_maxim()[0]
                specie_minima = specie
                genom_minim = specie.genom_maxim()[1]
        return minim
            
            
    def evolutie(self):
        pop_finala = self.determinare_populatie()
        lungime_populatie = [len(n.genomi) for n in self.spec]
        k = -1
        self.eliminare_underfit_specii(0)
        copii = []
        start_time = time.time()
        for specie in self.spec:
            k += 1
            ratio = specie.fitness_specie() / self.fitness_specii()
            offspring = math.floor(ratio * (pop_finala-lungime_populatie[k]) - 2)
            offspring = 11
            for j in range(0, int(offspring)):
                copii.append(specie.breed())
        print("Dupa evolutie:")
        print(time.time() - start_time)
        start_time = time.time()
        for copil in copii:
            self.clasificare_genom(copil)
        print("Dupa clasificare:")
        print(time.time() - start_time)
        start_time = time.time()
        for specie in self.spec:
            specie.generatie_curenta += 1
        print("Dupa modificare specie:")
        print(time.time() - start_time)

        
    
genab = Gena(0, 0)
gena1 = Gena(1, 0)
gena2 = Gena(2, 0)
gena3 = Gena(3, 2)
gena0 = Gena(5, 2)
gena4 = Gena(4, 1)
obj =  Conexiune(gena1, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
ob2 =  Conexiune(gena2, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
ob3 =  Conexiune(genab, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
ob4 =  Conexiune(gena1, gena3, random.uniform(0, 1)*random.choice([-1, 1]))
ob5 =  Conexiune(gena2, gena3, random.uniform(0, 1)*random.choice([-1, 1]))
ob6 =  Conexiune(genab, gena3, random.uniform(0, 1)*random.choice([-1, 1]))
ob7 =  Conexiune(gena3, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
ob8 = Conexiune(gena3, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
gen1 = Genom([genab,gena1, gena2, gena4], [obj, ob2, ob3])
gen2 = Genom([genab,gena1, gena2,gena3, gena4], [ob4, ob5, ob6, ob7])
gen3 = Genom([genab,gena1, gena2,gena3, gena4], [obj, ob2, ob4, ob7])
gen4 = Genom([genab,gena1, gena2, gena3, gena4], [obj, ob3, ob5, ob7])
gen5 = Genom([genab,gena1, gena2,gena3, gena4], [ob6, ob2, ob3, ob7])


nod_output = Gena(1,1)
nod_nou = Gena(2, 2)
bias = Gena(0, 0)
conn_bias = Conexiune(bias, nod_output, random.uniform(0, 1)*random.choice([-1, 1]))
conn_bias1 = Conexiune(bias, nod_nou, random.uniform(0, 1)*random.choice([-1, 1]))
conn_finala = Conexiune(nod_nou, nod_output, random.uniform(0, 1)*random.choice([-1, 1]))
maxim = 3
gene = [bias, nod_output]
gene1 = [bias, nod_output, nod_nou]
conexiuni = [conn_bias]
conexiuni1 = [conn_bias1, conn_finala]

for _ in range(0, 30):
    gena_noua = Gena(maxim,0)
    maxim += 1
    conn_noua = Conexiune(gena_noua, nod_output, random.uniform(0, 1)*random.choice([-1, 1]))
    conn_noua1 = Conexiune(gena_noua, nod_nou, random.uniform(0, 1)*random.choice([-1, 1]))
    conexiuni += [conn_noua]
    conexiuni1 += [conn_noua1]
    gene += [gena_noua]
    gene1 += [gena_noua]
    
genom_special = Genom(gene, conexiuni)
genom_special1 = Genom(gene1, conexiuni1)


lista1 = []
for _ in range(0, 15):
    #lista1.append(copy.deepcopy(random.choice([ genom_special])))
    lista1.append(copy.deepcopy(random.choice([genom_special])))
specie1= Specie(copy.deepcopy(lista1))
sp = []
for _ in range(0, 10):
    sp = sp + [copy.deepcopy(specie1)]
spc = Specii(copy.deepcopy(sp))

print(genom_special1)
            



Gene: 
0; 1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12; 13; 14; 15; 16; 17; 18; 19; 20; 21; 22; 23; 24; 25; 26; 27; 28; 29; 30; 31; 32; 
Conexiuni: 
0, 2, -0.3473922276252611, 9, 1 - ; 2, 1, 0.10458101367343986, 10, 1 - ; 3, 2, 0.42579100947562987, 12, 1 - ; 4, 2, 0.3128548589661301, 14, 1 - ; 5, 2, -0.32390993167356574, 16, 1 - ; 6, 2, 0.5919669751564419, 18, 1 - ; 7, 2, -0.5277092323051678, 20, 1 - ; 8, 2, 0.2576948272631735, 22, 1 - ; 9, 2, -0.48658373615537165, 24, 1 - ; 10, 2, 0.4912786455009397, 26, 1 - ; 11, 2, 0.08561837934170857, 28, 1 - ; 12, 2, 0.5761791874105834, 30, 1 - ; 13, 2, -0.5438604950220268, 32, 1 - ; 14, 2, -0.1891751159021683, 34, 1 - ; 15, 2, 0.7490516283080056, 36, 1 - ; 16, 2, 0.35235968756016234, 38, 1 - ; 17, 2, -0.028225476216828915, 40, 1 - ; 18, 2, -0.3092126126654068, 42, 1 - ; 19, 2, 0.6481640183770679, 44, 1 - ; 20, 2, 0.612642066877702, 46, 1 - ; 21, 2, 0.5083545337416542, 48, 1 - ; 22, 2, -0.2370620846038599, 50, 1 - ; 23, 2, -0.9666391781292145, 52, 1 - ; 

In [11]:
import time




precedent = 1000
epoci = 150
for i in range(0, len(spc.spec)):
    for genom in spc.spec[i].genomi:
        genom.fitness = genom.feed_forwoard()
actual = 100
while actual > 0.08:
    spc.eliminare_specii()
    actual = min([n.genom_maxim()[0] for n in spc.spec])
    print(actual)
    print(epoci)
    print([len(n.genomi) for n in spc.spec])
    #print([len(n.genomi) for n in spc.spec])
    start_time = time.time()
    epoci -= 1
    if precedent >= actual:
        for i in range(0, len(spc.spec)):
            for genom in spc.spec[i].genomi:
                genom.mutare()
    for i in range(0, len(spc.spec)):
        for genom in spc.spec[i].genomi:
            genom.fitness = genom.feed_forwoard()
    spc.evolutie()
    precedent = actual
    print("--- %s seconds iteratie---" % (time.time() - start_time))
for i in range(0, len(spc.spec)):
    for genom in spc.spec[i].genomi:
        genom.eliminare_disabled()
        genom.fitness = genom.feed_forwoard()

0.9858548085089314
150
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
0.8561012744903564
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 1.2614810466766357 seconds iteratie---
0.7000903540015984
149
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.006500244140625
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 1.9409425258636475 seconds iteratie---
0.17407851316949197
148
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.4404220581054688
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 2.645777463912964 seconds iteratie---
0.1723315411930275
147
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.1733386516571045
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 2.3630213737487793 seconds iteratie---
0.17102993561068697
146
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.1997976303100586
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 2.5269827842712402 seconds iteratie---
0.16720890896769083
145
[15, 15, 15, 15, 15, 

Dupa evolutie:
1.7248878479003906
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.059690237045288 seconds iteratie---
0.11950533356613849
107
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.8380556106567383
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 3.9933321475982666 seconds iteratie---
0.11947788908934706
106
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.019690752029419
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.8655006885528564 seconds iteratie---
0.11722636236471667
105
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.8091990947723389
Dupa clasificare:
0.0
Dupa modificare specie:
0.0009975433349609375
--- 4.0638885498046875 seconds iteratie---
0.11723885549272944
104
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.9550879001617432
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.160783290863037 seconds iteratie---
0.11723640212075272
103
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.86665225

Dupa evolutie:
2.0984368324279785
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.698292016983032 seconds iteratie---
0.10307435902922199
64
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.2349350452423096
Dupa clasificare:
0.0009396076202392578
Dupa modificare specie:
0.0
--- 4.745782852172852 seconds iteratie---
0.10306013713157142
63
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.8358609676361084
Dupa clasificare:
0.008050203323364258
Dupa modificare specie:
0.0
--- 4.1728515625 seconds iteratie---
0.10195025430013815
62
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.7634167671203613
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 5.233835458755493 seconds iteratie---
0.10193145245898086
61
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.1671853065490723
Dupa clasificare:
0.0009953975677490234
Dupa modificare specie:
0.0
--- 5.244487047195435 seconds iteratie---
0.10166585961230439
60
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Du

Dupa evolutie:
1.952406406402588
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.426318883895874 seconds iteratie---
0.09376135295534467
21
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.1164710521698
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.659900665283203 seconds iteratie---
0.09363564340346761
20
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.817138433456421
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.416712284088135 seconds iteratie---
0.09302508662259433
19
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.3889122009277344
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 5.458468914031982 seconds iteratie---
0.0930006631541358
18
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.109376907348633
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.895353555679321 seconds iteratie---
0.09290410526740593
17
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.2100682258605957
Dupa clasificare:
0.000

Dupa evolutie:
2.1494476795196533
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.875252962112427 seconds iteratie---
0.0889166232417803
-22
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.8649208545684814
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.36499285697937 seconds iteratie---
0.08859804485004315
-23
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.264956474304199
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.744745254516602 seconds iteratie---
0.08804445633367677
-24
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.2437744140625
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 5.431057691574097 seconds iteratie---
0.08807502045825645
-25
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.0794677734375
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.622189283370972 seconds iteratie---
0.08806633870740883
-26
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.8817517757415771
Dupa clasificare:
0.

Dupa evolutie:
1.9889299869537354
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.691305875778198 seconds iteratie---
0.08552571077105614
-65
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.204524040222168
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.799276828765869 seconds iteratie---
0.08552779807668641
-66
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.9503364562988281
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 5.02979850769043 seconds iteratie---
0.08552736214054518
-67
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.9622876644134521
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.63841700553894 seconds iteratie---
0.08523847385058297
-68
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.115478277206421
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.6640403270721436 seconds iteratie---
0.08449236174192636
-69
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.5867092609405518
Dupa clasifica

Dupa evolutie:
2.316478967666626
Dupa clasificare:
0.0009975433349609375
Dupa modificare specie:
0.0
--- 5.272420406341553 seconds iteratie---
0.08063974669073695
-108
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.8977975845336914
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.5478293895721436 seconds iteratie---
0.08062626229066425
-109
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.3238728046417236
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.829623460769653 seconds iteratie---
0.08063022129304383
-110
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.9077746868133545
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.475799560546875 seconds iteratie---
0.08062961235192706
-111
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
1.8730857372283936
Dupa clasificare:
0.0
Dupa modificare specie:
0.0
--- 4.584900379180908 seconds iteratie---
0.08048868497284999
-112
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
Dupa evolutie:
2.00312

In [11]:
print([n.genom_maxim()[0] for n in spc.spec])

print(spc.spec[6].genom_maxim()[1])

[0.12239561363135801, 0.11580571675327307, 0.10868033272188773, 0.12375359197348974, 0.11940126009963162, 0.08512260651017264, 0.0797213998784038, 0.09568854171277882, 0.10761203593456324, 0.11227943440372311]
Gene: 
0; 1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12; 13; 14; 15; 16; 17; 18; 19; 20; 21; 22; 23; 24; 25; 26; 27; 28; 29; 30; 31; 32; 
Conexiuni: 
0, 1, -3.836076997926935, 8, 1 - ; 2, 1, -4.494298896121613, 9, 1 - ; 3, 1, -0.030864325036469964, 10, 1 - ; 4, 1, -1.1318784552654366, 11, 1 - ; 5, 1, 0.6928692310229282, 12, 1 - ; 6, 1, -0.5853987232177026, 13, 1 - ; 7, 1, 2.2701356589126473, 14, 1 - ; 8, 1, 0.7258268590433298, 15, 1 - ; 9, 1, 0.08561401497073406, 16, 1 - ; 10, 1, 0.7788123668099238, 17, 1 - ; 11, 1, -0.10834835453525082, 18, 1 - ; 12, 1, 6.764046633465976, 19, 1 - ; 13, 1, -8.907547541343428, 20, 1 - ; 14, 1, 6.0914687012721975, 21, 1 - ; 16, 1, -1.0077677754288041, 23, 1 - ; 17, 1, 1.6859088417756825, 24, 1 - ; 18, 1, 2.086197441962028, 25, 1 - ; 19, 1, 1.27269801573442

In [42]:
print(specie1.distanta_genomi(genom_special, genom_special))

1


In [10]:
import numpy as np
import cv2



image = cv2.imread('C:\\Users\\MrSpV\\Desktop\\cancer1.jpg') #The function to read from an image into OpenCv is imread()

#imshow() is the function that displays the image on the scre0en.
#The first value is the title of the window, the second is the image file we have previously read.
#cv2.imshow("OpenCV Image Reading", image)
print(len(image) * len(image[0]))

44775


In [30]:
from matplotlib.image import imread
image = imread('C:\\Users\\MrSpV\\Desktop\\cancer2.jpeg')
lst = []
for i in range(1, len(image)):
    lst = lst + list(image[i])
        
print(len(lst))


44352


In [2]:
nod_output = Gena(1,1)
bias = Gena(0, 0)
conn_bias = Conexiune(bias, nod_output, random.uniform(0, 1)*random.choice([-1, 1]))
maxim = 2
gene = [bias, nod_output]
conexiuni = [conn_bias]
for _ in range(0, 30):
    gena_noua = Gena(maxim,0)
    maxim += 1
    conn_noua = Conexiune(gena_noua, nod_output, random.uniform(0, 1)*random.choice([-1, 1]))
    conexiuni += [conn_noua]
    gene += [gena_noua]
genom_special = Genom(gene, conexiuni)
print(genom_special.ff([1]*31))

15.55635600639211


In [131]:
for i in range(0, len(spc.spec)):
    for genom in spc.spec[i].genomi:
        genom.mutare()

In [10]:
spc.evolutie()

In [24]:
start_time = time.time()
for i in range(0, len(spc.spec)):
    for genom in spc.spec[i].genomi:
        genom.fitness = genom.feed_forwoard()
print(time.time() - start_time)

KeyboardInterrupt: 

In [21]:
teste = []
with open('C:\\Users\\MrSpV\\Desktop\\data.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        if row[0] == 'id':
            continue
        if row[1] == 'M':
            teste = teste + [([1] + list(map(lambda x: float(x), row[2:])),[1])]
        else:
            teste = teste + [([1] + list(map(lambda x: float(x), row[2:])),[0])]
teste = teste[1:]
teste = teste[35:40]
suma = 0
for i in teste[0][0]:
    suma += i * random.uniform(0, 1)*random.choice([-1, 1]) * 0.001 


print(teste)

[([1, 14.25, 21.72, 93.63, 633.0, 0.09823, 0.1098, 0.1319, 0.05598, 0.1885, 0.06125, 0.286, 1.019, 2.657, 24.91, 0.005878, 0.02995, 0.04815, 0.01161, 0.02028, 0.004022, 15.89, 30.36, 116.2, 799.6, 0.1446, 0.4238, 0.5186, 0.1447, 0.3591, 0.1014], [1]), ([1, 13.03, 18.42, 82.61, 523.8, 0.08983, 0.03766, 0.02562, 0.02923, 0.1467, 0.05863, 0.1839, 2.342, 1.17, 14.16, 0.004352, 0.004899, 0.01343, 0.01164, 0.02671, 0.001777, 13.3, 22.81, 84.46, 545.9, 0.09701, 0.04619, 0.04833, 0.05013, 0.1987, 0.06169], [0]), ([1, 14.99, 25.2, 95.54, 698.8, 0.09387, 0.05131, 0.02398, 0.02899, 0.1565, 0.05504, 1.214, 2.188, 8.077, 106.0, 0.006883, 0.01094, 0.01818, 0.01917, 0.007882, 0.001754, 14.99, 25.2, 95.54, 698.8, 0.09387, 0.05131, 0.02398, 0.02899, 0.1565, 0.05504], [1]), ([1, 13.48, 20.82, 88.4, 559.2, 0.1016, 0.1255, 0.1063, 0.05439, 0.172, 0.06419, 0.213, 0.5914, 1.545, 18.52, 0.005367, 0.02239, 0.03049, 0.01262, 0.01377, 0.003187, 15.53, 26.02, 107.3, 740.4, 0.161, 0.4225, 0.503, 0.2258, 0.2807, 0

In [103]:
def sigmoid(x):
    x = np.array([x])
    return np.mean(1/(1+ np.exp(-4.9 * x)))


In [105]:
def sigmoid1(x):
    return 1 / (1 + math.exp(-4.9 * x))